## Prepare count files

Install package based on [here](https://github.com/kauralasoo/rasqual/tree/master/rasqualTools)
using `devtools::install_github("kauralasoo/rasqual/rasqualTools")`. This has to be done in R console opened on a real terminal window. I installed it in conda env `testother`.

In [1]:
library(rasqualTools)
library(tibble)
library(dplyr)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
# Prepare count file
countfile <- "/project2/xinhe/ATAC-seq_10252018/CN/newCN_peaks.20_samples.counts.txt"
countm <- read.table(countfile, header=T,stringsAsFactors = F)
head(countm)

chr,start,end,peak_name,CN_02,CN_03,CN_04,CN_05,CN_06,CN_07,⋯,CN_12,CN_13,CN_14,CN_15,CN_16,CN_17,CN_18,CN_19,CN_20,CN_21
chr1,9909,10719,newCN1,583,676,637,568,643,978,⋯,751,560,948,793,546,739,1079,669,722,321
chr1,11093,11418,newCN2,51,110,57,24,60,132,⋯,63,129,98,51,52,49,93,20,97,49
chr1,19856,20555,newCN3,25,45,130,56,71,34,⋯,57,71,93,168,46,94,56,51,68,49
chr1,20674,20992,newCN4,7,26,35,16,17,16,⋯,25,22,30,44,18,21,32,18,32,22
chr1,28683,30094,newCN5,396,363,483,456,224,404,⋯,422,566,349,284,374,337,644,426,542,481
chr1,31250,31463,newCN6,16,28,36,24,31,26,⋯,16,40,27,23,33,31,31,43,23,19


In [3]:
dim(countm)

[1] 194079     24

In [4]:
counts_matrix <- countm[,5:dim(countm)[2]]
rownames(counts_matrix) <-countm$peak_name
head(counts_matrix)

,CN_02,CN_03,CN_04,CN_05,CN_06,CN_07,CN_08,CN_09,CN_10,CN_11,CN_12,CN_13,CN_14,CN_15,CN_16,CN_17,CN_18,CN_19,CN_20,CN_21
newCN1,583,676,637,568,643,978,653,673,986,529,751,560,948,793,546,739,1079,669,722,321
newCN2,51,110,57,24,60,132,126,53,125,86,63,129,98,51,52,49,93,20,97,49
newCN3,25,45,130,56,71,34,90,33,86,76,57,71,93,168,46,94,56,51,68,49
newCN4,7,26,35,16,17,16,34,15,36,18,25,22,30,44,18,21,32,18,32,22
newCN5,396,363,483,456,224,404,515,344,475,428,422,566,349,284,374,337,644,426,542,481
newCN6,16,28,36,24,31,26,48,21,32,31,16,40,27,23,33,31,31,43,23,19


In [8]:
saveRasqualMatrices(list("cellTypeCN" = counts_matrix), "../../datarun/CN", file_suffix = "atac20")

[1] "../../datarun/CN/cellTypeCN.atac20.txt"


In [9]:
# size factor file
size_factors = rasqualCalculateSampleOffsets(counts_matrix, gc_correct = FALSE)
saveRasqualMatrices(list("cellTypeCN" = size_factors), "../../datarun/CN", file_suffix = "size_factors")

[1] "../../datarun/CN/cellTypeCN.size_factors.txt"


In [10]:
# count SNP number per feature and generate batch run script.
gene_data =  as_data_frame(dplyr::select(countm, peak_name,chr,start,end))
gene_data = add_column(gene_data,strand= as.integer(1),.before="start")
colnames(gene_data) <- c("gene_id", "chr", "strand","exon_starts","exon_ends")
gene_data <- gene_data %>% mutate(exon_starts = as.character(exon_starts)) %>% mutate(exon_ends = as.character(exon_ends))

In [11]:
print(gene_data[1:5,])

# A tibble: 5 x 5
  gene_id chr   strand exon_starts exon_ends
  <chr>   <chr>  <int> <chr>       <chr>    
1 newCN1  chr1       1 9909        10719    
2 newCN2  chr1       1 11093       11418    
3 newCN3  chr1       1 19856       20555    
4 newCN4  chr1       1 20674       20992    
5 newCN5  chr1       1 28683       30094    


In [12]:
gen_command <- function(chrom, gdf, vcfdir, outdir, outtag) {
    qstart <- min(as.integer(gdf$exon_starts))
    qend <- max(as.integer(gdf$exon_ends))
    tmp_file <- tempfile(pattern = paste(chrom,qstart,qend,sep="_"), tmpdir = tempdir(), fileext = ".snplist")
    system(paste0("tabix ",vcfdir,"/",chrom,".hg38withchr.AS.vcf.gz ",chrom,":",qstart,"-",qend, "| cut -f 1-3> ", tmp_file))
    snpdf <- try(read.table(tmp_file, header=F, stringsAsFactors = F))
    if(inherits(snpdf, "try-error")){
        print(paste0("No line for ",chrom, outtag,". No output .sh file."))
        return(0)
    }
    file.remove(tmp_file)
    snpdf <- as_data_frame(snpdf)
    colnames(snpdf) <- c("chr", "pos", "snp_id")
    snp_counts <- countSnpsOverlapingExons(gdf, snpdf, cis_window = 2e3)
    cmd_mainfn <- paste0(outdir,"/","mainrun_",chrom, "_", outtag)
    cmd_permfn <- paste0(outdir,"/","permrun_",chrom, "_", outtag)
    mainbashf<-file(paste0(cmd_mainfn,".sh"),"w")
    permbashf<-file(paste0(cmd_permfn,".sh"),"w")
    writeLines(paste0("rm ", cmd_mainfn, ".result.txt"), mainbashf)
    writeLines(paste0("rm ",cmd_permfn, ".permres.txt"), permbashf)
    for (rowi in 1:nrow(snp_counts)){
        featureID <- snp_counts$gene_id[rowi]
        rSNP <- snp_counts$cis_snp_count[rowi]
        fSNP <- snp_counts$feature_snp_count[rowi]
        rstart <- snp_counts$range_start[rowi]
        rend <- snp_counts$range_end[rowi]
        fstart <- snp_counts$exon_starts[rowi]
        fend <- snp_counts$exon_ends[rowi]
        cmd_main <- paste0("tabix ",vcfdir,"/",chrom,".hg38withchr.AS.vcf.gz ",chrom,":", rstart, "-",rend,
                       "|/home/simingz/run_rasqual/rasqual/bin/rasqual -y ",outdir,"/cellTypeCN.atac20.bin -k ",
                       outdir, "/cellTypeCN.size_factors.bin -n 20 -j ",featureID," -l ",rSNP," -m ",fSNP,
                       " -s ",fstart," -e ", fend, " -f ", featureID," -z")
        cmd_perm <- paste0(cmd_main, " -r -t |  cut -f 1,10-12,25") # keep lead SNP only, select column
        writeLines(paste0(cmd_main," >> ",cmd_mainfn, ".result.txt"), mainbashf)
        writeLines(paste0("for i in `seq 1 1000`; do ", cmd_perm," >> ",cmd_permfn, ".permres.txt; done" ), permbashf)
    }
    close(mainbashf)
    close(permbashf)
    return(snp_counts)
}

In [13]:
nlinesperjob <- 220
vcfdir <- "/home/simingz/run_rasqual/phased_data"
datarundir <- "/home/simingz/run_rasqual/datarun/CN"

gene_data_bychr <- split(gene_data,gene_data$chr)
gene_data_bychr$chrX <- NULL
gene_data_bychr$chrY <- NULL

for (chrom in names(gene_data_bychr)){
    n <- nrow(gene_data_bychr[[chrom]])
    r <- rep(1:ceiling(n/nlinesperjob),each=nlinesperjob)[1:n]
    chromlist <- split(gene_data_bychr[[chrom]],r)
    for (tag in names(chromlist)){
        genedf <- chromlist[[tag]]
        gen_command(chrom,genedf,vcfdir,datarundir, paste0("CN",tag))
    }
}

[1] "No line for chr1CN43. No output .sh file."
[1] "No line for chr1CN44. No output .sh file."
[1] "No line for chr1CN45. No output .sh file."
[1] "No line for chr1CN46. No output .sh file."
[1] "No line for chr1CN47. No output .sh file."
[1] "No line for chr1CN48. No output .sh file."
[1] "No line for chr1CN49. No output .sh file."
[1] "No line for chr10CN12. No output .sh file."
[1] "No line for chr10CN13. No output .sh file."
[1] "No line for chr13CN1. No output .sh file."
[1] "No line for chr13CN2. No output .sh file."
[1] "No line for chr13CN3. No output .sh file."
[1] "No line for chr13CN4. No output .sh file."
[1] "No line for chr20CN9. No output .sh file."
[1] "No line for chr6CN19. No output .sh file."
[1] "No line for chr9CN14. No output .sh file."
